# FIT3179 Data Visualisation 2

Source: https://www.kaggle.com/datasets/alphajuliet/au-dom-traffic?resource=download
Origionally Sourced From: https://data.gov.au/dataset/ds-dga-c5029f2a-39b3-4aef-8ae1-73e7962f6170/details as Domestic Airlines - Top Routes (CSV)

Working with a single data set file containing aggregate details on Australian Domestic Flights each month from Jan-1984 to Aug-2021.

### Verbatim Notes Provided by Original Source

The data cover revenue passengers carried by Australian-registered operators of scheduled regular public transport services over Australian flight stages. 
These estimates include passengers carried between domestic airports on international flights operated by these carriers.
The passengers carried refer to traffic on board by stages.
Traffic on board by stages - The total of all traffic (revenue passengers) on each flight stage between two directly connected airports.

Individual routes shown are restricted to those with an average exceeding 8 000 passengers per month over the previous six months where two or more airlines operate in competition. For the month of March 2016 there were 66 such routes. 
Cells with zero indicate where data is not available or where the route did not meet the publication criteria above.

**Definitions:**
* Aircraft Trips - The number of flight stages. A return flight counts as two aircraft trips.
* Available Seat Kilometres (ASKs) - Calculated by multiplying the number of seats available on each flight stage, by the distance in kilometres between the ports. The distances used are Great Circle Distances.
* City-Pair - The ports shown make up the city-pair route. Passenger movements shown for a city pair reflect total traffic in both directions.
* Distance - Great Circle Distances between two airports.
* Flight Stage - The operation of an aircraft from take-off to landing.
* Passengers Carried - Revenue passengers carried.
* Passenger Load Factor - The total revenue passenger kilometres performed as a percentage of the total available seat kilometres.
* Regular Public Transport Services (RPT) - All air service operations in which aircraft are available for the transport of members of the public and are conducted in accordance with fixed schedules. It does not include charter or other non-scheduled operations.
* Revenue Passengers - All passengers paying any fare. Frequent flyer redemption travellers are regarded as revenue passengers.
* Revenue Passenger Kilometres (RPKs) - Calculated by multiplying the number of revenue passengers travelling on each flight stage, by the distance in kilometres between the ports. The distances used are Great Circle Distances.

### Column Definitions
* City1: The first part of the origin, destination pair. Note the city1 and city2 values show data for flights in both directions.
* City2: The second part of the origin, destination pair.
* Month: The month-year of the flight data shown.
* Passenger Count: The number of passengers flown between the two cities in the given month.
* Flight Count: The number of flights between the two cities in the given month (in either direction).
* Average Capacity (%): How full the plane is in terms of passenger seating capacity. Average value for the month.
* Flight Distance (km): Distance between airports (using Great Circle distance). Is not the actual distance flow.
* Actual Passenger Distance (km): Flight Distance * Passenger Count
* Potential Passenger Distance (km): Flight Distance * Total Seat Available
* Seats: The total number of seats available on flights for the given month between the two locations.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import csv, os, random, math

In [2]:
# 1000 Seperator
def format_large_numbers(x):
    return '{:,}'.format(x)

# Set the custom formatting function
pd.options.display.float_format = format_large_numbers

In [3]:
path = 'Data/raw_data.csv'
data = pd.read_csv(path)
# Rename the columns
new_names = {
    'Passenger_Trips': 'Passenger Count',
    'Aircraft_Trips': 'Flight Count',
    'Passenger_Load_Factor': 'Average Capacity (%)',
    'Distance_GC_(km)': 'Flight Distance (km)'
}
data.rename(columns=new_names, inplace=True)
# Add field for plane size (in terms of seat count)
data['Plane Size (seats)'] = data['Seats'] / data['Flight Count']
# Remove fields not needed
data.drop(columns=['Month_num', 'RPKs', 'ASKs', 'Seats'], inplace=True)


# Removing Temporal Nature
# Aggregate the data to yearly
data = data.groupby(['Year', 'City1', 'City2'], as_index=False).agg({
    'Passenger Count': 'sum', 'Flight Count': 'sum', 'Average Capacity (%)': 'mean', 
    'Flight Distance (km)': 'mean', 'Plane Size (seats)': 'mean'
})
# Include only 2017 non-zero data (it has the most connections)
data = data[(data['Year']==2017) & (data['Passenger Count'] > 0)]
# Remove the now redundant Year column
data.drop(columns=['Year'], inplace=True)
# Round all numbers to 2dp
data = data.applymap(lambda x: round(x, 2) if isinstance(x, float) else x)


In [4]:
# Get Number of connections for each location
cities = data['City1'].append(data['City2']).unique()
num_connections = {city: 0 for city in cities}
for _, row in data.iterrows():
    num_connections[row['City1']] += 1
    num_connections[row['City2']] += 1

# Rearrange City1 and City2 to have City1 be the the more connected city (where possible)
def rearrange_route(row):
    if num_connections[row['City1']] < num_connections[row['City2']]:
        row['City1'], row['City2'] = row['City2'], row['City1']
    return row
data = data.apply(rearrange_route, axis=1)
data = data.sort_values(by='City1', key=lambda x: data['City1'].map(num_connections), ascending=False)
data = data.reset_index(drop=True)


## Secondary Dataset
There is a secondary dataset which contains location data for each of the cities. Latitude, Longitude, Population, Area.

This dataset was sourced from Chat-GPT (Sep/25 Version).
Prompts:
* 'I am going to give a list of places in australia. I want a CSV response with the following data about each. Latitude, Longitude, Population, Area (km sq), State. The cities: ADELAIDE,ALBURY,ALICE SPRINGS,BRISBANE,BROOME,CANBERRA,COFFS HARBOUR,DARWIN,DEVONPORT,DUBBO,GOLD COAST,HOBART,KALGOORLIE,KARRATHA,LAUNCESTON,MELBOURNE,PERTH,SUNSHINE COAST,SYDNEY,CAIRNS,BALLINA,HERVEY BAY,PORT MACQUARIE,NEWCASTLE,HAMILTON ISLAND,AYERS ROCK,GERALDTON,BUNDABERG,ARMIDALE,PROSERPINE,MACKAY,ROCKHAMPTON,TOWNSVILLE,BURNIE,MILDURA,WAGGA WAGGA,PORT LINCOLN,PORT HEDLAND,NEWMAN,GLADSTONE,EMERALD,MOUNT ISA,MORANBAH,TAMWORTH'
* Give me the same data but: 1. Abbreviate the State Names, 2. Add a boolean (True, False) field 'Is State Capitol', 3. Add a field 'Airport distance to CBD (km).

### Defining 
Cities are labeled as Capitol, Metro, Regional. Regional Locations are defined as having a population less than 50,000.

In [8]:
#cols = ['Lat', 'Long', 'Pop', 'Area (km sq)', 'State', 'Is Capitol', 'Airport Distance to CBD']
data_locations = pd.read_csv('Data/location_data.csv')
data_locations['Connections'] = data_locations['City'].map(num_connections)
data_locations.drop(columns=['Area (km sq)', 'Airport distance to CBD (km)'], inplace=True)
data_locations = data_locations.sort_values(by=['Connections', 'Population'], 
                                            ascending=False).reset_index(drop=True)

### Adding Route Type

We now use `data_locations` to determine the route type for each row in `data`.

In [10]:
city_classes = data_locations.set_index('City')['Class'].to_dict()
def get_routetype(row):
    city1_class = city_classes[row['City1']]
    city2_class = city_classes[row['City2']]
    return f'{city1_class}-{city2_class}'
data['Route Type'] = data.apply(get_routetype, axis=1)

### Heatmap Dataset
Creating a third dataset specifically for the heatmap.
Has a more even spread of City1 and City2.

In [16]:
def get_state(city):
    return data_locations.loc[data_locations['City'] == city, 'State'].iloc[0]

states = list(data_locations['State'].unique())
combos = [[set(i), 0] for i in itertools.combinations_with_replacement(states, 2)]
for index, row in data.iterrows():
    state1, state2 = get_state(row['City1']), get_state(row['City2'])
    key = set([state1, state2])
    value = row['Passenger Count']
    for index, amount in enumerate(combos):
        if key == amount[0]:
            combos[index][1] += value

extra = []
for index, value in enumerate(combos):
    row = list(value[0])
    if len(row) == 1:
        row += row
        row.append(value[1])
        combos[index] = row
    else:
        row.append(value[1])
        combos[index] = row
        row_reverse = [row[1], row[0], row[2]]
        extra.append(row_reverse)
combos += extra
data_heatmap = pd.DataFrame(combos, columns=['State1', 'State2', 'Value'])

### Flights per City 

Adding Flights to the Data_Locations column.

In [17]:
def get_class(city):
    return data_locations.loc[data_locations['City'] == city, 'Class'].iloc[0]

cities = {i: 0 for i in data_locations['City'].unique()}
for index, route in data.iterrows():
    cities[route['City1']] += route['Flight Count']
    cities[route['City2']] += route['Flight Count']
cities = pd.Series(cities).reset_index()
cities = cities.rename(columns={'index': 'City', 0: 'Flights'})
data_locations = pd.merge(data_locations, cities, on='City', how='left')

### Get KPI Values

1. Number of Routes: 68
2. Number of Airports: 41
3. Number of People: 54,629,078
4. Number of Flights: 471,363

In [24]:
kpi_1 = len(data)
kpi_2 = len(data_locations)
kpi_3 = data['Passenger Count'].sum()
kpi_4 = data['Flight Count'].sum()
kpis = {'Num Routes': kpi_1, 'Num Airports': kpi_2, 'Num People': kpi_3, 'Num Flights': kpi_4}

In [25]:
kpis

{'Num Routes': 68,
 'Num Airports': 41,
 'Num People': 54629078,
 'Num Flights': 471363}

## Export

In [19]:
data.to_csv('Data/flight_data_cleaned.csv')
data_locations.to_csv('Data/location_data_cleaned.csv')
data_heatmap.to_csv('Data/heatmap_data.csv')